**A.     代码内容说明（启动训练及推理的方式、必要注释等）；**

启动训练，请看【4 训练】  
推理，请看【5 推理】

**B.     模型构建思路及调优过程（可具体包括：思路框架图、思路步骤详述、模型应用+调优过程）；**

调优过程主要是在赛题基线的基础上，调整了BatchSize与学习率进行训练。

**C.     若参考他人代码，请在比赛报告中附上链接以便审查。**

参考了赛题基线：
https://aistudio.baidu.com/aistudio/personalcenter/thirdview/277680

**checkpoint文件**  
checkpoint.zip

**提交结果**  
0523.zip

# 1 环境准备

In [1]:
# 解压PaddleSeg压缩包
!unzip -oq data/data88946/PaddleSeg.zip -d /home/aistudio/
# 修改文件名
!mv PaddleSeg-release-v2.0 PaddleSeg

mv: cannot move 'PaddleSeg-release-v2.0' to 'PaddleSeg/PaddleSeg-release-v2.0': Directory not empty


上一步mv，可以将PaddleSeg加压后的文件目录改成PaddleSeg
>PaddleSeg下载至github的release2.0版本，为了方便大家使用，已添加在了数据集中供大家使用

In [ ]:
# 删除data目录 —— 用于数据划分时，产生了意料之外的数据扩充时的数据重置
# %cd /home/aistudio/
# !rm -rf PaddleSeg/data
# 解压数据集到PaddleSeg目录下的data文件夹
!unzip -oq /home/aistudio/data/data93545/常规赛：PALM病理性近视病灶检测与分割.zip -d PaddleSeg/data

In [ ]:
# 查看数据集文件的树形结构
!tree -d PaddleSeg/data/常规赛：PALM病理性近视病灶检测与分割

通过PIL的Image读取图片查看以下原数据与Label标注情况

In [ ]:
from PIL import Image
import numpy as np
# 读取图片
png_img = Image.open('PaddleSeg/data/常规赛：PALM病理性近视病灶检测与分割/Train/fundus_image/H0003.jpg')
png_img  # 展示真实图片

In [ ]:
bmp_img = Image.open('PaddleSeg/data/常规赛：PALM病理性近视病灶检测与分割/Train/Lesion_Masks/Atrophy/H0003.bmp')
bmp_img   # 展示萎缩标注图片

In [ ]:
bmp_img = Image.open('PaddleSeg/data/常规赛：PALM病理性近视病灶检测与分割/Train/Lesion_Masks/Detachment/P0053.bmp')
bmp_img   # 展示脱落标注图片

# 2 划分数据集与数据预处置

当前划分比例为0.8——可在utils文件夹下的dataset_splited.py修改**train_percent**为其它值

数据预处置-可在utils文件夹下的dataset_pretrans.py中查看相关代码--实现将255转化为1，原问题变二分类问题

> 注意：当前数据处理中，存在对数据进行扩充，因此当前程序运行一次之后会使得原分类数据数目增加——(扩增不宜过大，否则划分数据进行训练和验证时会出现偏差问题)

> 仅可运行一次，多次运行会导致填充数量过多，这是源码中扩充机制决定的(直接扩充到源文件夹中，所以下一次划分的时候就会默认把源文件中所有的文件读取)

> 感兴趣可前往查看dataset_splited.py的数据扩充区

In [ ]:
# 保证路径为初始路径
%cd /home/aistudio

# 划分数据
!python utils/dataset_splited.py

# 转换标签--预处置
!python utils/dataset_pretrans.py

移除原数据，减小项目空间，减少下一次进入和退出保存时花的时间

In [ ]:
# 移除’常规赛：PALM病理性近视病灶检测与分割‘文件夹
!rm -rf PaddleSeg/data/常规赛：PALM病理性近视病灶检测与分割
!rm -rf PaddleSeg/data/__MACOSX 

# 3 选择比赛模型

基线模型为:  配置略微修改的`PaddleSeg/configs/emanet/emanet_resnet50_os8_voc12aug_512x512_40k.yml`

具体配置在
	
	-- example/emanet_resnet50_os8_voc12aug_512x512_40k_Deta.yml
    * 用于分割脱落情况
    
	-- example/emanet_resnet50_os8_voc12aug_512x512_40k_Atro.yml
    * 用于分割萎缩情况

详细数据集配置在

	-- example/pascal_voc2012_Deta.yml
    * 用于设置脱落情况数据

	-- example/pascal_voc2012_Atro.yml
    * 用于设置萎缩情况数据
    

> 针对不同分割任务配置不同的分割模型，以适配不同的任务驱动

> 将预置的基线配置yml移动到相应的文件夹下

* _base_: 数据加载yml存放

* emanet: emanet模型yml存放

In [ ]:
%cd /home/aistudio/
!cp -u example/pascal_voc2012_Atro.yml PaddleSeg/configs/_base_/
!cp -u example/pascal_voc2012_Deta.yml PaddleSeg/configs/_base_/
!cp -u example/emanet_resnet50_os8_voc12aug_512x512_40k_Deta.yml PaddleSeg/configs/emanet/
!cp -u example/emanet_resnet50_os8_voc12aug_512x512_40k_Atro.yml PaddleSeg/configs/emanet/


# 4 训练

In [ ]:
# 下载依赖项，保证PaddleSeg正常运行
%cd PaddleSeg
%pwd
!pip install -r requirements.txt

> 更换自定义的模型文件时，只需要修改对应的模型yml、数据集yml(_base_中的yml)、以及替换下方的yml即可进行训练了

> 不要更改输出目录，否则后边的代码也需要修改，包括预测、后处理中的所有路径

In [14]:
%cd PaddleSeg
%pwd
# 训练分割Detachment的模型，并保存到/output/Detachment
!python train.py --c configs/emanet/emanet_resnet50_os8_voc12aug_512x512_40k_Deta.yml \
--use_vdl \
--save_interval 100 \
--do_eval \
--seed 2021 \
--iters 1000 \
--learning_rate 0.01 \
--save_dir ./output/Detachment

# 训练分割Atrophy的模型，并保存到/output/Atrophy
!python train.py --c configs/emanet/emanet_resnet50_os8_voc12aug_512x512_40k_Atro.yml \
--use_vdl \
--save_interval 200 \
--do_eval \
--seed 2021 \
--iters 3000 \
--learning_rate 0.01 \
--save_dir ./output/Atrophy

# 5 推理

这可以直接使用emanet进行预测，不用修改数据集yml，也不用修改相应的模型yml，注意训练权重对应即可！

> 预测结果按类别分别放在`./output/result/Detachment` 和 `./output/result/Atrophy` 下

> 如果使用不同的模型对不同的分割类进行讨论，注意模型yml即可

> 提交结果为两种单独预测的结果

**默认使用Iou评估最好的模型训练参数--best_model**

In [15]:
%cd PaddleSeg/
# 预测Detachment
!python predict.py --config configs/emanet/emanet_resnet50_os8_voc12aug_512x512_40k_Deta.yml \
--model_path output/Detachment/best_model/model.pdparams \
--image_path data/Detachment/Test \
--save_dir ./output/result/Detachment

# 预测Atrophy
!python predict.py --config configs/emanet/emanet_resnet50_os8_voc12aug_512x512_40k_Atro.yml  \
--model_path output/Atrophy/best_model/model.pdparams \
--image_path data/Atrophy/Test \
--save_dir ./output/result/Atrophy

# 6 后处理

* 将类别值1换为255，进行赛题结果提交

In [16]:
%cd /home/aistudio/
!python utils/post_process.py

# 7 生成结果文件

In [17]:
# 复制文件到最顶层目录
%cd /home/aistudio
!cp -r PaddleSeg/output/result/ Lesion_Segmentation

# 过程移动文件--保证不包含生成的子目录
%cd Lesion_Segmentation
!cp -r Detachment/pseudo_color_prediction/. Detachment
!cp -r Atrophy/pseudo_color_prediction/. Atrophy

# 获取指定的提交目录格式
!rm -rf Detachment/added_prediction
!rm -rf Detachment/pseudo_color_prediction

!rm -rf Atrophy/added_prediction
!rm -rf Atrophy/pseudo_color_prediction

# 压缩文件
%cd /home/aistudio
!zip -r Lesion_Segmentation.zip Lesion_Segmentation
# 删除复制的文件
!rm -rf Lesion_Segmentation

In [ ]:
# # 删除zip的文件--丢失提交结果，需重新后处理生成
# !rm -rf Lesion_Segmentation.zip
# # 删除预测结果--丢失预测结果，需重新预测
# !rm -rf PaddleSeg/output/result
# # 删除output文件夹--丢失模型参数，需重新训练
# !rm -rf PaddleSeg/output
# # 删除data文件夹--数据将丢失，需要重新解压，划分，预处置
# !rm -rf PaddleSeg/data